# Deploying Iris-detection model using Vertex AI


## Overview

In this tutorial, you build a scikit-learn model and deploy it on Vertex AI using the custom container method. You use the FastAPI Python web server framework to create a prediction endpoint. You also incorporate a preprocessor from training pipeline into your online serving application.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this notebook, you learn how to create, deploy and serve a custom classification model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself. 


This tutorial uses the following Vertex AI services and resources:

- Vertex AI models
- Vertex AI endpoints

The steps performed include:

- Train a model that uses flower's measurements as input to predict the class of iris.
- Save the model and its serialized pre-processor.
- Build a FastAPI server to handle predictions and health checks.
- Build a custom container with model artifacts.
- Upload and deploy custom container to Vertex AI Endpoints.

### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small and popular dataset for machine learning experiments. Each instance has four numerical features, which are different measurements of a flower, and a target label that
categorizes the flower into: **Iris setosa**, **Iris versicolour** and **Iris virginica**.

This tutorial uses [a version of the Iris dataset available in the
scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing) and [Cloud Build pricing](https://cloud.google.com/build/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages



In [1]:

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

### Set Google Cloud project information 
Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "operating-realm-460905-q8"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [3]:
BUCKET_URI = f"gs://mlops-bucket-week1"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-bucket-week1/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-bucket-week1' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 

In [5]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [6]:
import os
import sys

### Configure resource names

Set a name for the following parameters:

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed to the repository.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI model resource.

In [7]:
MODEL_ARTIFACT_DIR = "my-models/iris-classifier-week-1"  # @param {type:"string"}
REPOSITORY = "iris-classifier-repo"  # @param {type:"string"}
IMAGE = "iris-classifier-img"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "iris-classifier"  # @param {type:"string"}

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-container-prediction-model"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-container-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"

## Simple Decision Tree model
Build a Decision Tree model on iris data

In [13]:
! mkdir data artifacts

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

data = pd.read_csv('iris.csv')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [14]:
train, test = train_test_split(data, test_size = 0.4, stratify = data['species'], random_state = 42)
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.species
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.species

In [15]:
mod_dt = DecisionTreeClassifier(max_depth = 3, random_state = 1)
mod_dt.fit(X_train,y_train)
prediction=mod_dt.predict(X_test)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))

The accuracy of the Decision Tree is 0.983


In [16]:
import pickle
import joblib

joblib.dump(mod_dt, "artifacts/model.joblib")

['artifacts/model.joblib']

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [17]:
# !gsutil cp artifacts/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      


In [16]:
# -- Step 4: Initialize DVC
!git init
!dvc init -f
!git add .dvc .gitignore
!git commit -m "Initialize DVC"

Reinitialized existing Git repository in /home/jupyter/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>
fatal: pathspec '.gitignore' did not match any files
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/

In [22]:
# environment setup
!python3 -m venv .env
!source .env/bin/activate
# install -r requirements.txt
!pip install dvc


In [23]:
# -- Step 5: Add iris dataset to DVC tracking
!dvc add data/iris.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
Adding...                                                                       
ERROR: unexpected error - no such column: "size" - should this be a string literal in single-quotes?

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [21]:
!git add data/iris.csv.dvc
!git commit -m "Track iris.csv with DVC"

⠋ Checking graph                                       core>
Adding...                                                                       
!
Adding...                                                                       
ERROR: unexpected error - no such column: "size" - should this be a string literal in single-quotes?

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
fatal: pathspec 'data/iris.csv.dvc' did not match any files
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/
	.docker/
	.gitconfig
	.gsutil/
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.local/
	.npm/
	SDK_Custom_Container_Prediction.ipynb
	artifacts/
	data/
	notebook_template.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
# -- Step 7: Tag this as version 1
!git tag v1
!git push origin --tags

In [ ]:
# -- Step 8: Make changes for version 2
# Example: Replace iris.csv with iris_v2.csv or update scripts
mv data/iris_v2.csv data/iris.csv
dvc add data/iris.csv
git add data/iris.csv.dvc
git commit -m "Version 2: Update dataset and pipeline changes"

In [ ]:
# -- Step 9: Push data and code changes
dvc push
git push

In [ ]:
# -- Step 10: Tag as version 2
git tag v2
git push origin --tags

In [ ]:
# -- Step 11: Demonstrate switching between versions
# To revert to version 1
git checkout v1
dvc pull

In [3]:
!pip install dvc


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 60.9 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144592 sha256=4383fb5eac245297cffd7b75d008a850f42b4e18c260b3bbb1b45d74b18fdd77
  Stored in directory: /home/jupyter/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [4]:
! dvc --version

3.60.1


In [5]:
!git init
!dvc init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/jupyter/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                             

In [8]:
!git add .dvc .gitignore
! git commit -m "Initialize DVC"

fatal: pathspec '.gitignore' did not match any files
[master (root-commit) 1901ded] Initialize DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


In [7]:
! git config --global user.email "21f2000243@ds.study.iitm.ac.in"
!git config --global user.name "satyamsaitama"

In [9]:
# -- Step 5: Add iris dataset to DVC tracking
!dvc add data/iris.csv
!git add data/iris.csv.dvc
!git commit -m "Track iris.csv with DVC"

⠋ Checking graph                                       core>
Adding...                                                                       
!
Adding...                                                                       
ERROR: unexpected error - no such column: "size" - should this be a string literal in single-quotes?

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
fatal: pathspec 'data/iris.csv.dvc' did not match any files
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/
	.docker/
	.gitconfig
	.gsutil/
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.local/
	.npm/
	SDK_Custom_Container_Prediction.ipynb
	artifacts/
	data/
	notebook_template.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [10]:
! git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.bashrc
	.cache/
	.config/
	.docker/
	.gitconfig
	.gsutil/
	.ipynb_checkpoints/
	.ipython/
	.jupyter/
	.local/
	.npm/
	SDK_Custom_Container_Prediction.ipynb
	artifacts/
	data/
	notebook_template.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [11]:
# -- Step 7: Tag this as version 1
!git tag v1
!git push origin --tags

fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [13]:
# -- Step 8: Make changes for version 2
# Example: Replace iris.csv with iris_v2.csv or update scripts
mv data/iris_v2.csv data/iris.csv
dvc add data/iris.csv
git add data/iris.csv.dvc
git commit -m "Version 2: Update dataset and pipeline changes"

In [13]:
# -- Step 9: Push data and code changes
dvc push
git push

In [13]:
# -- Step 10: Tag as version 2
git tag v2
git push origin --tags

In [13]:
# -- Step 11: Demonstrate switching between versions
# To revert to version 1
git checkout v1
dvc pull

error: pathspec 'main' did not match any file(s) known to git


# FEAST [WEEK 3]

In [20]:
!pip install feast scikit-learn 'feast[gcp]' --quiet

In [21]:
!pip install "numpy<2" "pandas==2.2.2" --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.2 requires pandas<2.2.0, but you have pandas 2.2.2 which is incompatible.
sktime 0.26.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.2.2 which is incompatible.


In [5]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from google.cloud import bigquery

In [21]:
df = pd.read_csv('data/iris.csv')

In [22]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [23]:
df["flower_id"] = np.arange(len(df))
df["event_timestamp"] = pd.date_range("2021-01-01", periods=len(df), freq="h")
df["created"] = datetime.utcnow()

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,flower_id,event_timestamp,created
0,5.1,3.5,1.4,0.2,setosa,0,2021-01-01 00:00:00,2025-06-22 18:01:51.054597
1,4.9,3.0,1.4,0.2,setosa,1,2021-01-01 01:00:00,2025-06-22 18:01:51.054597
2,4.7,3.2,1.3,0.2,setosa,2,2021-01-01 02:00:00,2025-06-22 18:01:51.054597
3,4.6,3.1,1.5,0.2,setosa,3,2021-01-01 03:00:00,2025-06-22 18:01:51.054597
4,5.0,3.6,1.4,0.2,setosa,4,2021-01-01 04:00:00,2025-06-22 18:01:51.054597


In [24]:
df.shape

(150, 8)

In [10]:
ls

SDK_Custom_Container_Prediction.ipynb  iris_feature_repo/
artifacts/                             logs.log
data/                                  notebook_template.ipynb*
feast-driver-ranking-tutorial/


In [25]:
PROJECT_ID = "operating-realm-460905-q8"
BUCKET_NAME= "mlops-bucket-week3"
BIGQUERY_DATASET_NAME = "iris_dataset_week3"
AI_PLATFORM_MODEL_NAME = "iris_table_jsd_model"

In [26]:
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

!gcloud config set project {PROJECT_ID} --quiet
!echo project_id = {PROJECT_ID} > ~/.bigqueryrc


Updated property [core/project].


In [27]:
client = bigquery.Client(project=PROJECT_ID)

# Create dataset if not exists
dataset_ref = bigquery.DatasetReference(PROJECT_ID, BIGQUERY_DATASET_NAME)
try:
    client.get_dataset(dataset_ref)
except:
    client.create_dataset(bigquery.Dataset(dataset_ref), timeout=30)

# Upload the DataFrame
client.load_table_from_dataframe(
    df, f"{PROJECT_ID}.{BIGQUERY_DATASET_NAME}.iris_table"
).result()


LoadJob<project=operating-realm-460905-q8, location=US, id=a0e867f1-eda7-4323-90dd-4ad49e866630>

In [18]:
!feast init iris_feature_repo

/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/types/Value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/DataFormat.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/Feature.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/pytho

In [28]:
%cd iris_feature_repo/feature_repo

/home/jupyter/iris_feature_repo/feature_repo


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
df.to_parquet("data/iris.parquet", index=False)


In [30]:
!feast apply

/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/types/Value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/DataFormat.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/Feature.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/pytho

In [20]:
!echo ".ipynb_checkpoints/" >> .feastignore

In [31]:
from feast import FeatureStore

store = FeatureStore(repo_path=".")

start_date = datetime(2021, 1, 1)
end_date = datetime.utcnow()

store.materialize(start_date=start_date, end_date=end_date)

/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/types/Value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/DataFormat.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at feast/core/Feature.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/conda/lib/pytho

Materializing 0 feature views from 2021-01-01 00:00:00+00:00 to 2025-06-22 18:03:55+00:00 into the sqlite online store.



/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


In [33]:
online_features_df = store.get_online_features(
    features=[
        "iris_stats_view:sepal_length",
        "iris_stats_view:sepal_width",
        "iris_stats_view:petal_length",
        "iris_stats_view:petal_width",
        "iris_stats_view:species"
    ],
    entity_rows=[{"sample_id": i} for i in range(0, 150)]
).to_df()

print("Online features:")
print(online_features_df.head())

Online features:
   flower_id  petal_width  petal_length species  sepal_length  sepal_width
0          0          0.2           1.4  setosa           5.1          3.5
1          1          0.2           1.4  setosa           4.9          3.0
2          2          0.2           1.3  setosa           4.7          3.2
3          3          0.2           1.5  setosa           4.6          3.1
4          4          0.2           1.4  setosa           5.0          3.


In [ ]:
local_df = df.copy()
local_df["sample_id"] = range(len(local_df))
encoder = OrdinalEncoder()
local_df["target"] = encoder.fit_transform(local_df[["species"]])


In [ ]:
# Merge online and local data if you wish to enrich features or labels
combined_df = pd.concat([online_features_df, local_df['target']], axis=1)

combined_df.drop('species', inplace=True, axis=1)

In [34]:
combined_df.head()

,flower_id,petal_width,petal_length,sepal_length,sepal_width,target
0,0,0.2,1.4,5.1,3.5,0.0
1,1,0.2,1.4,4.9,3.0,0.0
2,2,0.2,1.3,4.7,3.2,0.0
3,3,0.2,1.5,4.6,3.1,0.0
4,4,0.2,1.4,5.0,3.6,0.0


In [35]:
X = combined_df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = combined_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

print("Accuracy:", accuracy_score(y_test, model.predict(X_test)))

Accuracy: 1.0


In [36]:
import joblib
import os

# Save locally first
os.makedirs("../..artifacts", exist_ok=True)
joblib.dump(model, "../..artifacts/iris_model.joblib")

# Upload to GCS
!gsutil cp ../..artifacts/iris_model.joblib gs://$BUCKET_NAME/models/


Copying file://../..artifacts/iris_model.joblib [Content-Type=application/octet-stream]...
/ [1 files][187.2 KiB/187.2 KiB]                                                 
Operation completed over 1 objects/187.2 KiB.


# Week 4

### Install Pytest

In [4]:
! pip install pytest

# Initialize Git and Branches

In [32]:
!git init
!git remote add origin https://github.com/SatyamSaitama/iris-pipeline.git
!git checkout -b dev


Reinitialized existing Git repository in /home/jupyter/.git/
Switched to a new branch 'dev'


In [33]:
! sudo apt install tree # for seeing structure

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (1.8.0-1+b1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [34]:
! tree .

.
├── SDK_Custom_Container_Prediction.ipynb
├── __pycache__
│   └── test_train.cpython-310-pytest-8.4.1.pyc
├── artifacts
│   └── model.joblib
├── data
│   └── iris.csv
├── feast-driver-ranking-tutorial
│   ├── README.md
│   ├── driver_orders.csv
│   ├── driver_ranking
│   │   ├── data
│   │   │   └── registry.db
│   │   ├── driver_repo.py
│   │   └── feature_store.yaml
│   ├── notebooks
│   │   └── Driver_Ranking_Tutorial.ipynb
│   ├── predict.py
│   └── train.py
├── iris_feature_repo
│   ├── README.md
│   ├── __init__.py
│   └── feature_repo
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-310.pyc
│       │   ├── example_repo.cpython-310.pyc
│       │   └── test_workflow.cpython-310.pyc
│       ├── data
│       │   ├── driver_stats.parquet
│       │   ├── iris.parquet
│       │   ├── online_store.db
│       │   └── registry.db
│       └── feature_store.yaml
├── logs.log
├── notebook_template.ipynb
├── pytest.ini
├── src
│   ├── __init__.py
│   ├── __py

In [20]:
! pytest test_train.py


============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.4.1, pluggy-1.5.0
rootdir: /home/jupyter
plugins: typeguard-4.4.2, dash-3.0.4, hydra-core-1.3.2, anyio-4.9.0
collected 4 items                                                              

test_train.py ....                                                       [100%]

============================== 4 passed in 2.82s ===============================


`Create evaluation and data validation unit tests using pytest`

In [21]:
# ! touch src/evaluate.py src/validate.py

In [22]:
# ! touch tests/test_evaluate.py tests/test_validate.py

### This tells pytest to include the current directory (which should contain src/) in the Python path.



In [28]:
#Commit and Push
!touch pytest.ini
!pytest

============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.4.1, pluggy-1.5.0
rootdir: /home/jupyter
configfile: pytest.ini
plugins: typeguard-4.4.2, dash-3.0.4, hydra-core-1.3.2, anyio-4.9.0
collected 6 items                                                              

test_train.py ....                                                       [ 66%]
tests/test_evaluate.py .                                                 [ 83%]
tests/test_validate.py .                                                 [100%]

============================== 6 passed in 1.29s ===============================


In [36]:
! pytest tests/test_validate.py
! pytest tests/test_evaluate.py

============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.4.1, pluggy-1.5.0
rootdir: /home/jupyter
configfile: pytest.ini
plugins: typeguard-4.4.2, dash-3.0.4, hydra-core-1.3.2, anyio-4.9.0
collected 1 item                                                               

tests/test_validate.py .                                                 [100%]

============================== 1 passed in 0.58s ===============================
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.4.1, pluggy-1.5.0
rootdir: /home/jupyter
configfile: pytest.ini
plugins: typeguard-4.4.2, dash-3.0.4, hydra-core-1.3.2, anyio-4.9.0
collected 1 item                                                               

tests/test_evaluate.py .                                                 [100%]

============================== 1 passed in 1.19s ===============================


#### Commit and Push

> Run a **sanity test using GitHub Actions**, printing a report **as a comment using CML**

##### Add GitHub Action Workflow with CML

#####`.github/workflows/sanity.yml`


In [37]:
!git add .
!git commit -m "Add evaluation and validation tests"
!git push origin dev


hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> feast-driver-ranking-tutorial
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached feast-driver-ranking-tutorial
hint: 
hint: See "git help submodule" for more information.
[dev ba97166] Add evaluation and validation tests
 2487 files changed, 315851 insertions(+)
 create mode 100644 .bashrc
 create mode 100644 .bigqueryrc
 create mode 100644 .cache/jedi/CPython-310-33/4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-349f2ce27ea32e4072f92dbeec012d3898ee6ce80aff1962bdd10be515f67b14.pkl
 create mode 100644 .cache/jedi/CPython-310-33/4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-a37ba278a3d2d16c2fc9